# Shor's order-finding algorithm
This notebook exemplifies using [Quaspy](https://github.com/ekera/quaspy) to simulate Shor's order-finding algorithm [[Shor94]](https://doi.org/10.1109/SFCS.1994.365700), modified as in [[E22p]](https://doi.org/10.48550/arXiv.2201.07791), and with the classical post-processing in [[E22p]](https://doi.org/10.48550/arXiv.2201.07791).

To start off, let us pick an $m$-bit order $r$ uniformly at random from the set of all $m$-bit integers.

To end, we use the [<code>sample_l_bit_integer(l)</code>](../docs/math/random/sample_l_bit_integer.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy):

In [5]:
from quaspy.math.random import sample_l_bit_integer;

m = 1024;
r = sample_l_bit_integer(m);

print("Sampled r =", r);

Sampled r = 121678063190985161654857727149697826498910236313799625365809752509418153535888625119669691903583093187478255432771526335823812715768555120223756800507731079213173237628903312270742598015398249763393869936425760315264701801540661843871342631205235593297575469599572539289653228843979106011279889347614951025057


## 1. Simulating the quantum part of Shor's order-finding algorithm to sample $j$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_j_given_r(r, m, l, ..)</code>](../docs/orderfinding/general/sampling/sample_j_given_r.md) for simulating Shor's order-finding algorithm exactly for a given order $r$.

Below, we use this function to simulate running the algorithm for $r$ with a control register of $m + \ell$ bits.

This yields a frequency $j$ sampled from the probability distribution induced by the algorithm:

In [6]:
from quaspy.orderfinding.general.sampling import sample_j_given_r;

l = m; # Ensures that r^2 < 2^(m + l) as r < 2^m;

j = sample_j_given_r(r, m, l);

print("Sampled j =", j);

Sampled j = 16555691887535788270739670307963792383449484644283632165386040484577198565812326213844089889603027458859295265983127684970932297733407674744325014302505309127679731551740158467515497715986205493953518031091990485736970704924925615177674257749680180071766404281876043317711725768781415441812609454798685917879310041330486674511710941165462622084666073206513529436348594567818863022251291843329750506734479497086509781288897497815455535859803605798793622227718840650371170845393998371978107888425780214620729082211252160252166283236548939202800246780004898209075172732115914109036783262772384586494968762270514073467154


### 1.1. Solving $j$ for $r$

We now proceed to solve $j$ for $r$.

To this end, we first use the [SimulatedCyclicGroupElement](../docs/math/groups/SimulatedCyclicGroupElement.md) class provided by [Quaspy](https://github.com/ekera/quaspy) to define a group element $g$ of order $r$:

In [7]:
from quaspy.math.groups import SimulatedCyclicGroupElement;

g = SimulatedCyclicGroupElement(r);

We may then solve $j$ for $r$ by calling the [<code>solve_j_for_r(j, m, l, g, ..)</code>](../docs/orderfinding/general/postprocessing/ekera/solve_j_for_r.md) function provided by [Quaspy](https://github.com/ekera/quaspy).

By default this function uses the lattice-based post-processing from [[E22p]](https://doi.org/10.48550/arXiv.2201.07791) without enumerating the lattice, in combination with searching offsets in $j$, and reconstructing the missing part of the order when smooth, and so forth, see [[E22p]](https://doi.org/10.48550/arXiv.2201.07791) for the full details and a lower bound on the success probability.

This yields the order $r$:

In [8]:
from quaspy.orderfinding.general.postprocessing.ekera import solve_j_for_r;

result = solve_j_for_r(j, m, l, g);

print("Solving for r yielded:", result);

if result == r:
  print("\n[ OK ] The order r was successfully recovered.");
else:
  print("\n[FAIL] The order r was not recovered.");

Solving for r yielded: 121678063190985161654857727149697826498910236313799625365809752509418153535888625119669691903583093187478255432771526335823812715768555120223756800507731079213173237628903312270742598015398249763393869936425760315264701801540661843871342631205235593297575469599572539289653228843979106011279889347614951025057

[ OK ] The order r was successfully recovered.
